In [207]:
import pandas as pd
from bs4 import BeautifulSoup
import os

In [208]:
df = pd.read_csv('bestofrt.tsv', sep='\t')

In [235]:
df['title'] = df['title'].astype("string")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   ranking                   100 non-null    int64 
 1   critic_score              100 non-null    int64 
 2   title                     100 non-null    string
 3   number_of_critic_ratings  100 non-null    int64 
dtypes: int64(3), string(1)
memory usage: 2.8 KB


In [236]:
with open('rt-html/rt_html/et_the_extraterrestrial.html') as file:
    soup = BeautifulSoup(file, 'lxml');
    
soup.find('title')

<title>E.T. The Extra-Terrestrial (1982) - Rotten Tomatoes</title>

In [237]:
#splicing to grab movie title
soup.find('title').contents[0][:-len(' - Rotten Tomatoes')]

'E.T. The Extra-Terrestrial\xa0(1982)'

In [238]:
x = soup.find('div', class_='audience-info hidden-xs superPageFontColor')
x = x.find_all('div')[1].contents[2].strip()
x

'32,313,030'

In [239]:
# List of dictionaries to build file by file and later convert to a DataFrame
df_list = []
folder = 'rt-html/rt_html'
for movie_html in os.listdir(folder):
    with open(os.path.join(folder, movie_html)) as file:
        soup = BeautifulSoup(file, 'lxml')
        title = soup.find('title').contents[0][:-len(' - Rotten Tomatoes')]
        audience_score = soup.find('div', class_='audience-score meter').find('span').contents[0][:-1]
        num_of_audience_ratings = soup.find('div', class_='audience-info hidden-xs superPageFontColor')
        num_of_audience_ratings = num_of_audience_ratings.find_all('div')[1].contents[2].strip().replace(',', '')
        
        # Append to list of dictionaries
        df_list.append({'title': title,
                        'audience_score': int(audience_score),
                        'number_of_audience_ratings': int(num_of_audience_ratings)})
df_audience = pd.DataFrame(df_list, columns = ['title', 'audience_score', 'number_of_audience_ratings'])

In [240]:
df_audience['title'] = df_audience['title'].astype("string")
df_audience.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   title                       100 non-null    string
 1   audience_score              100 non-null    int64 
 2   number_of_audience_ratings  100 non-null    int64 
dtypes: int64(2), string(1)
memory usage: 2.0 KB


In [245]:
df2 = pd.merge(df_audience, df, on='title')

In [246]:
df2.head()

,title,audience_score,number_of_audience_ratings,ranking,critic_score,number_of_critic_ratings
